In [23]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


#Creating Helper functions python file


In [24]:
%%writefile helper.py

#importing libraries
import cv2
import numpy as np
import torch
import matplotlib
import matplotlib.pyplot as plt
import pandas as pd

char = '01234567ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz'

#1
def img_cleaning(img):
  #converting the image to grayscale
  img_gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)

  #applied gaussian blur to reduce noise
  img_blur = cv2.GaussianBlur(img_gray, (0,0), sigmaX=4, sigmaY=4)

  #thresholding the image
  dst = cv2.adaptiveThreshold(img_blur, 255,
	              cv2.ADAPTIVE_THRESH_MEAN_C, cv2.THRESH_BINARY_INV, 21, 6)

  return dst


#2 contour detection and extracting the images
def read(dst):

  #dilating the image
  dilated = cv2.dilate(dst.copy(), None, iterations=14)

  #detecting contours
  contours, hierarchy = cv2.findContours(dilated, mode=cv2.RETR_EXTERNAL, method=cv2.CHAIN_APPROX_SIMPLE)

  #removing small contours
  for c in contours:
    area = cv2.contourArea(c)

    # Small contours are ignored.
    if area < 5000:
        cv2.fillPoly(dilated, pts=[c], color=0)
        continue
  contours, hier = cv2.findContours(dilated, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)

  #Arranging contours in order
  m = []
  for i in range (len(contours)):
    c_0 = contours[i]

    # Get the 4 points of the bounding rectangle
    x, y, w, h = cv2.boundingRect(c_0)
    m.append([x,y,w,h])
  m = sorted(m, key = lambda x:x[0])
  captcha = []

  #list of extracted letters
  for i in m:
    image = dst[i[1]:i[1]+i[3] , i[0]:i[0]+i[2]]
    captcha.append(image)

  #reshaping these and appending them to a list
  IMG_HEIGHT = 28
  IMG_WIDTH = 28
  for i in range(len(captcha)):
    captcha[i]=cv2.resize(captcha[i], (IMG_HEIGHT, IMG_WIDTH),interpolation = cv2.INTER_AREA)

  return captcha


#3
def output(data,model):
  device = 'cuda' if next(model.parameters()).is_cuda else 'cpu'
  cap = ''
  for i in range(len(data)):
    t = torch.tensor(data[i],dtype = torch.float32)
    t = t.reshape(1,1,28,28)
    t = t.to(device)
    outputs = model(t)
    predicted = torch.max(outputs.data , 1)[1]
    cap += char[predicted.item()]
  return cap

Overwriting helper.py


#Creating app.py for web app

In [25]:
! pip install streamlit -q

In [26]:
!wget -q -O - ipv4.icanhazip.com

35.233.172.44


In [27]:
%%writefile app.py
import torch
import torch.nn as nn
import torch.nn.functional as F
import streamlit as st
import cv2
from PIL import Image, ImageOps
import numpy as np
from helper import *

class Net(nn.Module):
        def __init__(self):
            super(Net, self).__init__()
            self.conv1 = nn.Conv2d(1, 32, 5)
            self.conv2 = nn.Conv2d(32, 32, 5)
            self.conv3 = nn.Conv2d(32, 64, 4)
            self.pool = nn.MaxPool2d(2, 2)
            self.fc1 = nn.Linear(64, 256)
            self.fc2 = nn.Linear(256, 128)
            self.fc3 = nn.Linear(128, 64)
            self.fc4 = nn.Linear(64, 60)

        def forward(self, x):
            x = self.pool(F.relu(self.conv1(x)))
            x = self.pool(F.relu(self.conv2(x)))
            x = self.conv3(x)
            x = torch.flatten(x, 1)
            x = F.relu(self.fc1(x))
            x = F.relu(self.fc2(x))
            x = F.relu(self.fc3(x))
            x = F.relu(self.fc4(x))
            return x

def load_model(PATH):
    # Define device and load model
    device = 'cuda' if torch.cuda.is_available() else 'cpu'
    print(device)

    # Load the model, specifying map_location='cpu'
    model = torch.load(PATH, map_location=torch.device('cpu'))
    return model

PATH = '/content/drive/MyDrive/Handwritten-Captcha-Solver/model.h5'  # path of model
loaded_model = load_model(PATH)

st.write(""" # Handwritten Captcha Solver """ )
file = st.file_uploader("🔒 Crack the Code", type=["jpg", "png","jpeg"])

if file is None:
    st.info("Upload a captcha image above to generate the prediction.")
else:
    image = Image.open(file)
    st.image(image, use_column_width=True)

    # Convert PIL Image to numpy array
    img_array = np.array(image)

    # Clean the image
    c_img = img_cleaning(img_array)

    # Read data from the cleaned image
    data = read(c_img)

    # Generate the captcha output
    predicted_captcha = output(data,loaded_model)

    # Define a dictionary mapping numbers to emojis
    emoji_dict = {
        '1': '✅',
        '2': '☁️',
        '3': '🥐',
        '4': '❤️',
        '5': '😄',
        '6': '😊',
        '7': '☀️'
    }

    # Replace numbers with emojis in the predicted_captcha string
    for num, emoji in emoji_dict.items():
        predicted_captcha = predicted_captcha.replace(num, emoji)

    captcha = "Predicted Captcha is : " + predicted_captcha

    st.success(captcha)

Overwriting app.py


In [28]:
!streamlit run app.py & npx localtunnel --port 8501

[..................] - fetchMetadata: sill resolveWithNewModule localtunnel@2.0


  You can now view your Streamlit app in your browser.

  Network URL: http://172.28.0.12:8501
  External URL: http://35.233.172.44:8501

npx: installed 22 in 3.421s
your url is: https://true-rice-yell.loca.lt
cuda
cuda
  Stopping...
^C
